<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/symmetryfin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.half().to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics
import math
import os
from torchvision import models, transforms
from PIL import Image
from sklearn.linear_model import RANSACRegressor
deeplab_model = models.segmentation.deeplabv3_resnet101(pretrained=True).to(device).eval()

preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def symmetry_calc(left, right):
    si = abs(left - right) / (0.5 * (left + right))

    return {
        "symmetry": round(si, 3)
    }


def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def camera_to_pixel_coords(X, Y, Z, f_x, f_y, c_x, c_y):
    u = int((X * f_x) / Z + c_x)
    v = int((Y * f_y) / Z + c_y)
    return u, v

def point_to_plane_distance(point, plane_model):
    a, b, c, d = plane_model
    X, Y, Z = point
    return abs(a*X + b*Y + c*Z + d) / np.sqrt(a**2 + b**2 + c**2)

def median_depth_patch(depth_map, u, v, radius=3):
    """Return median depth around pixel (u,v). radius in pixels."""
    h, w = depth_map.shape
    u0 = max(0, u-radius); u1 = min(w-1, u+radius)
    v0 = max(0, v-radius); v1 = min(h-1, v+radius)
    patch = depth_map[v0:v1+1, u0:u1+1].ravel()
    patch = patch[np.isfinite(patch) & (patch > 0)]
    if len(patch)==0:
        return None
    return float(np.median(patch))

def fit_local_ground_plane_3d(depth_map, ground_mask, fx, fy, cx, cy,
                              u_center, v_center, window=80, sample_step=4, min_pts=50):
    """
    Fit plane in camera coordinates using ground_mask *near* (u_center,v_center).
    Returns plane (a,b,c,d) for ax + by + cz + d = 0 and list of sampled 3D points used.
    """
    h, w = depth_map.shape
    u0 = max(0, int(u_center - window))
    u1 = min(w-1, int(u_center + window))
    v0 = max(0, int(v_center - window))
    v1 = min(h-1, int(v_center + window))

    us, vs = np.meshgrid(np.arange(u0, u1+1, sample_step),
                         np.arange(v0, v1+1, sample_step))
    us = us.ravel(); vs = vs.ravel()

    mask_vals = ground_mask[vs, us]
    zs = depth_map[vs, us]
    keep = mask_vals & np.isfinite(zs) & (zs > 0.01)
    if keep.sum() < min_pts:
        return None, None  # not enough local ground points

    us_k = us[keep]; vs_k = vs[keep]; zs_k = zs[keep]
    # backproject to camera coords
    Xs = (us_k - cx) * zs_k / fx
    Ys = (vs_k - cy) * zs_k / fy
    Zs = zs_k
    pts3 = np.vstack([Xs, Ys, Zs]).T

    # fit Z = a*X + b*Y + c  via RANSAC (robust)
    XY = pts3[:, :2]
    Zvals = pts3[:, 2]
    ransac = RANSACRegressor(residual_threshold=0.03, min_samples=20, max_trials=200)
    ransac.fit(XY, Zvals)
    a, b = ransac.estimator_.coef_
    c = float(ransac.estimator_.intercept_)
    # plane: Z = aX + bY + c  ->  aX + bY - Z + c = 0
    # get normalized (A,B,C,D)ر
    A, B, C, D = a, b, -1.0, c
    norm = np.array([A, B, C], dtype=float)
    norm /= np.linalg.norm(norm)
    D = D / np.linalg.norm([A,B,C])
    return (norm[0], norm[1], norm[2], D), pts3

def processvideo(video_path, output_path,model):
    left=0
    right=0
    S_I=0
    vr = VideoReader(video_path)
    frame_count = len(vr)
    print(f"Total frames: {frame_count}")
    fps = 7
    step = fps * 1
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=True)
    mp_draw = mp.solutions.drawing_utils
    sample_frame = vr[0].asnumpy()
    rgb = cv2.cvtColor(sample_frame , cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    img1 = transform(rgb).unsqueeze(0).to(device).half()
    height, width, _ = sample_frame.shape
    c_x,c_y= approximate_intrinsics(width, height)
    with torch.no_grad():
            pred = model.infer(img1)
            depth_map = pred["depth"].squeeze().cpu().numpy()
            depth_map = cv2.resize(depth_map, (sample_frame.shape[1], sample_frame.shape[0]))
            intrinsics = pred["focallength_px"].cpu().numpy()
            if np.ndim(intrinsics) == 0:
              fx = fy = float(intrinsics)
            else:
              fx, fy = intrinsics
    print("kk")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for i in range(0, frame_count, step):
        frame = vr[i].asnumpy()
        rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)

        #ectracting ground
        img = transform(rgb).unsqueeze(0).to(device)
        deeplab_input = preprocess(rgb).unsqueeze(0).to(device)

        with torch.no_grad():
           out1 = deeplab_model(deeplab_input)['out'][0]
           seg = out1.argmax(0).cpu().numpy()

        ground_mask = (seg == 0) | (seg == 2)

        points = {}
        mp_pose = mp.solutions.pose
        h, w, _ = frame.shape
        if results.pose_landmarks:
           for id, lm in enumerate(results.pose_landmarks.landmark):
             cx= min(max(int(lm.x * w), 0), w - 1)
             cy= min(max(int(lm.y * h), 0), h - 1)
             cz = median_depth_patch(depth_map, cx, cy, radius=3)

             X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
             if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
                points["left_ankle"] = (X, Y, Z)
             elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
                points["right_ankle"] = (X, Y, Z)
           u_ankle1 = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].x * width)
           v_ankle1 = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE].y * height)
           u_ankle2 = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].x * width)
           v_ankle2 = int(results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE].y * height)


           plane1, pts31 = fit_local_ground_plane_3d(depth_map, ground_mask, fx, fy, cx, cy,u_center=u_ankle1, v_center=v_ankle1,window=100, sample_step=4)
           plane2, pts32 = fit_local_ground_plane_3d(depth_map, ground_mask, fx, fy, cx, cy,u_center=u_ankle2, v_center=v_ankle2,window=100, sample_step=4)
           distl = point_to_plane_distance(points["left_ankle"], plane1)
           distr= point_to_plane_distance(points["right_ankle"], plane2)
           print(f"distl: {distl}")
           print(f"distr: {distr}")


           if distr>distl and (distr- distl) > 0.05:
            print(distl)
            left= left+1
            left_t= 'left'
            right_t= 'none'
           elif distl>distr and (distl- distr) > 0.05:
            print(distr)
            right=right+1
            right_t= 'right'
            left_t= 'none'
           else:
            left_t= 'none'
            right_t= 'none'

           text = f"Frame: {i} | left: {distl:.4f} | right: {distr:.4f} | touching: {left_t if left_t!='none' else right_t}"

# Choose font and scale
           font = cv2.FONT_HERSHEY_SIMPLEX
           font_scale = 2.0
           thickness = 3

# Get text size (width, height)
           (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)

# If text too wide for frame, reduce font_scale
           if text_w > frame.shape[1] - 20:  # keep margin
               font_scale = (frame.shape[1] - 10) / (0.5*text_w)

# Place text at safe position (top-left but inside)
           x = 10
           y = 50 + text_h

           cv2.putText(frame, text, (x, y), font, font_scale, (0, 0, 200), thickness, cv2.LINE_AA)


        out.write(frame)
    if left > 0 or right !=0:
       S_I= symmetry_calc(left,right)
       print(S_I)
    black_frame = np.zeros_like(frame)
    for _ in range(fps * 3):

         frame_with_text = black_frame.copy()

         txt = f"{S_I}"
         txt1 = f"left: {left}"
         txt2 = f"right: {right}"

         font = cv2.FONT_HERSHEY_SIMPLEX
         font_scale = 2.0
         thickness = 3
         color = (255, 255, 255)


         (text_width, text_height), baseline = cv2.getTextSize(txt, font, font_scale, thickness)
         (text_width1, text_height1), baseline = cv2.getTextSize(txt1, font, font_scale, thickness)
         (text_width2, text_height2), baseline = cv2.getTextSize(txt2, font, font_scale, thickness)


         x = (width - text_width) // 2
         y = (height + text_height) // 2
         x1 = (width - text_width1) // 2
         y1 = (height + 4* text_height1) // 2
         x2 = (width - text_width2) // 2
         y2 = (height + 8* text_height2) // 2


         cv2.putText(frame_with_text,
            txt,
            (x, y),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)
         cv2.putText(frame_with_text,
            txt1,
            (x1, y1),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)
         cv2.putText(frame_with_text,
            txt2,
            (x2, y2),
            font,
            font_scale,
            color,
            thickness,
            cv2.LINE_AA)

         out.write(frame_with_text)
    out.release()


input_folder = "/content/drive/MyDrive/test"
output_folder = "/content/drive/MyDrive/testr"
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".mp4", ".avi", ".mov"))]

for video in video_files:
    input_path = os.path.join(input_folder, video)
    output_name = os.path.splitext(video)[0] + "_fin.mp4"
    output_path = os.path.join(output_folder, output_name)

    print(f"Processing {video} ...")
    processvideo(input_path, output_path,model)
    print(f"Saved result to {output_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing test11.mp4 ...
Total frames: 226
kk
distl: 0.49413335089910615
distr: 0.4601693929919157
distl: 0.5008755131301627
distr: 0.4108482558382889
0.4108482558382889
distl: 0.08629604599964313
distr: 0.16419748259818734
0.08629604599964313
distl: 0.21933691555686807
distr: 0.2790652907825957
0.21933691555686807
distl: 0.13489151209186545
distr: 0.012760869611400882
0.012760869611400882
distl: 0.2901275276729032
distr: 0.02795819781986675
0.02795819781986675
distl: 0.2320383945712583
distr: 0.24505128879975582
distl: 0.22982584626076433
distr: 0.05731589995258091
0.05731589995258091
distl: 0.20794155550225746
distr: 0.11819988362983441
0.11819988362983441
distl: 0.0075377420283773144
distr: 0.04976102202544586
distl: 0.21274804821016782
distr: 0.6216789157262909
0.21274804821016782
distl: 0.16355317825372143
distr: 0.14846772172275746
distl: 0.1232654855749904
distr: 0.13439007798560199
distl: 0.05092749610595715
distr: 0.07476060101268067
distl: 0.16236315599901485
distr: 0.127521

In [2]:
!pip install mediapipe
!pip install decord


!pip install git+https://github.com/apple/ml-depth-pro.git
!mkdir -p checkpoints
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O checkpoints/depth_pro.pt

  Cloning https://github.com/apple/ml-depth-pro.git to /tmp/pip-req-build-4ymbblp4
  Running command git clone --filter=blob:none --quiet https://github.com/apple/ml-depth-pro.git /tmp/pip-req-build-4ymbblp4
  Resolved https://github.com/apple/ml-depth-pro.git to commit 9efe5c1def37a26c5367a71df664b18e1306c708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 42.2 MB/s eta 0:00:00
  Created wheel for depth_pro: filename=depth_pro-0.1-py3-none-any.whl size=27593 sha256=5e0f24c24125da3a57565c73c5d38206e47aaad6c57fc77e3437ef42ec7accab
  Stored in directory: /tmp/pip-ephem-wheel-cache-9q8uvtsu/wheels/3f/ad/4e/a2b790772df19931cab8bf9c5af51665ae6502f39589e662c4
Successfully built depth_pro
--2025-09-10 19:21:34--  https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.34, 13.35